In [30]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from collections import OrderedDict

In [62]:
class Test(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.seq = nn.Sequential(OrderedDict({
            'linear1': nn.Linear(1, 2), 
            'ReLU1': nn.ReLU(),
            'Linear2': nn.Linear(2, 2)}))
        
    def forward(self, x):
        x = self.seq(x)
        return x
    
t = Test()
x = torch.tensor(1.0, requires_grad=True)
optimizer = torch.optim.Adam(t.parameters())

In [63]:
list(t.named_parameters())[0]

('seq.linear1.weight',
 Parameter containing:
 tensor([[-0.4277],
         [-0.5577]], requires_grad=True))

In [67]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.tensor = torch.tensor(1.0, requires_grad=True)
        self.param = nn.Parameter(torch.tensor(2.0))
        self.register_parameter('manual', nn.Parameter(torch.tensor(2.0)))
        self.register_parameter('2', self.param)
        self.buf2 = nn.Buffer(torch.tensor(3.0), persistent=False)
        
        self.buffer = nn.parameter.Buffer(torch.tensor(3.0))
        self.submodule = nn.Linear(2, 4)
        
    def forward(self):
        print('666')

In [68]:
my = Model()

In [70]:
for i in my.named_modules():
    print(i)
    print()

('', Model(
  (submodule): Linear(in_features=2, out_features=4, bias=True)
))

('submodule', Linear(in_features=2, out_features=4, bias=True))



In [59]:
my.buf

tensor(1.)

In [22]:
for name, param in my.named_parameters():
    print(name)
    print(param)

param
Parameter containing:
tensor(2., requires_grad=True)
manual
Parameter containing:
tensor(2., requires_grad=True)
submodule.weight
Parameter containing:
tensor([[ 0.5277, -0.6729],
        [-0.3889, -0.3282],
        [-0.1269,  0.5668],
        [ 0.4076, -0.5337]], requires_grad=True)
submodule.bias
Parameter containing:
tensor([-0.1100, -0.0587, -0.4846,  0.3501], requires_grad=True)


In [45]:
for param in my.parameters():
    print(type(param), param.size())

<class 'torch.nn.parameter.Parameter'> torch.Size([])
<class 'torch.nn.parameter.Parameter'> torch.Size([])
<class 'torch.nn.parameter.Parameter'> torch.Size([4, 2])
<class 'torch.nn.parameter.Parameter'> torch.Size([4])


In [75]:
class MM(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.linear = nn.Linear(2, 4)
        self.my = Model()

In [76]:
mm = MM()

In [79]:
for i in mm.named_modules(prefix='MM'):
    print(i)

('MM', MM(
  (linear): Linear(in_features=2, out_features=4, bias=True)
  (my): Model(
    (submodule): Linear(in_features=2, out_features=4, bias=True)
  )
))
('MM.linear', Linear(in_features=2, out_features=4, bias=True))
('MM.my', Model(
  (submodule): Linear(in_features=2, out_features=4, bias=True)
))
('MM.my.submodule', Linear(in_features=2, out_features=4, bias=True))


In [78]:
for i in mm.named_children():
    print(i)

('linear', Linear(in_features=2, out_features=4, bias=True))
('my', Model(
  (submodule): Linear(in_features=2, out_features=4, bias=True)
))


In [84]:
for i in mm.named_parameters():
    print(i)
    print()

('linear.weight', Parameter containing:
tensor([[-0.0606, -0.1771],
        [-0.1730, -0.1171],
        [-0.2045,  0.5385],
        [ 0.6317, -0.2190]], requires_grad=True))

('linear.bias', Parameter containing:
tensor([-0.6745,  0.4009,  0.4866,  0.2634], requires_grad=True))

('my.param', Parameter containing:
tensor(2., requires_grad=True))

('my.manual', Parameter containing:
tensor(2., requires_grad=True))

('my.submodule.weight', Parameter containing:
tensor([[-0.4292,  0.3409],
        [ 0.5350, -0.6695],
        [ 0.3177, -0.4256],
        [-0.6050, -0.1262]], requires_grad=True))

('my.submodule.bias', Parameter containing:
tensor([ 0.5736,  0.4845, -0.5701,  0.3365], requires_grad=True))



In [89]:
mm.apply(lambda x: print(x._get_name()))

Linear
Linear
Model
MM


MM(
  (linear): Linear(in_features=2, out_features=4, bias=True)
  (my): Model(
    (submodule): Linear(in_features=2, out_features=4, bias=True)
  )
)

In [91]:
class A(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.A_param = nn.Parameter(torch.tensor(1.0))
        self.A_Linear = nn.Linear(1, 2)
        
class B(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.B_param = nn.Parameter(torch.tensor(1.0))
        self.B_RNN = nn.RNN(1, 2)
        self.B_A = A()
        

In [92]:
b = B()

In [98]:
c = b.apply(lambda x: print(x, '\n'))

RNN(1, 2) 

Linear(in_features=1, out_features=2, bias=True) 

A(
  (A_Linear): Linear(in_features=1, out_features=2, bias=True)
) 

B(
  (B_RNN): RNN(1, 2)
  (B_A): A(
    (A_Linear): Linear(in_features=1, out_features=2, bias=True)
  )
) 



In [96]:
for i in b.named_children():
    print(i)

('B_RNN', RNN(1, 2))
('B_A', A(
  (A_Linear): Linear(in_features=1, out_features=2, bias=True)
))


In [103]:
b.state_dict()

OrderedDict([('B_param', tensor(1.)),
             ('B_RNN.weight_ih_l0', tensor([[0.0687],
                      [0.1565]])),
             ('B_RNN.weight_hh_l0',
              tensor([[ 0.3150, -0.5467],
                      [-0.2539, -0.5615]])),
             ('B_RNN.bias_ih_l0', tensor([-0.1778,  0.2578])),
             ('B_RNN.bias_hh_l0', tensor([0.2587, 0.0184])),
             ('B_A.A_param', tensor(1.)),
             ('B_A.A_Linear.weight',
              tensor([[ 0.9959],
                      [-0.0425]])),
             ('B_A.A_Linear.bias', tensor([-0.5509,  0.6909]))])